In [ ]:
import json
import time
import os
import praw

# Reddit API credentials
reddit = praw.Reddit(
    client_id="bgpEcNkIWPdnxbMZ2sO8uw",
    client_secret="9dNxl_O_ZAjqFCwDEUGISoxBiDr9yQ",
    user_agent="PythonAPI",
)

# Configuration
SUBREDDIT_NAME = "lonely"  # Change to any subreddit
#SUBREDDIT_NAME = "LonelyTogether"
OUTPUT_FILE = "C:/Users/w065pxg/Desktop/ML/Tanvi Banerjee 2024 MI/Final Project/StudentResearch/RedditData.json"
PULL_LIMIT = 250 # Number of posts to pull
KEYWORD = ""  # Leave empty to fetch all posts
RATE_LIMIT_DELAY = 1.5  # Adjust this based on Reddit's API rate limits

# Load existing data
flair_data, meta_data, pulled_ids = {}, {}, set()
if os.path.exists(OUTPUT_FILE):
    try:
        with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
            data = json.load(f)
            flair_data = data.get('flair', {})
            meta_data = data.get('meta', {})
            pulled_ids = set(meta_data.keys())
            print(f"📂 Loaded {len(pulled_ids)} existing posts.")
    except json.JSONDecodeError:
        print("⚠️ Existing file is empty or corrupted. Starting fresh.")

# Connect to subreddit
subreddit = reddit.subreddit(SUBREDDIT_NAME)

# Initialize counters
count_posts, count_duplicates, count_skipped = 0, 0, 0

print(f"🚀 Fetching up to {PULL_LIMIT} posts from r/{SUBREDDIT_NAME}...\n")

try:
    for post in subreddit.new(limit=PULL_LIMIT):
        if count_posts >= PULL_LIMIT:
            break

        if post.id in pulled_ids:
            count_duplicates += 1
            continue  # Skip duplicates

        if post.num_comments == 0:
            count_skipped += 1
            continue  # Skip posts without comments

        # Extract relevant details
        flair = post.link_flair_text or "No Flair"
        upvotes = post.ups  # Number of upvotes
        num_comments = post.num_comments  # Number of comments

        # Print post details in console
        print(f"✔️ {count_posts+1}. {post.title} (⬆️ {upvotes} upvotes, 💬 {num_comments} comments)")

        # Store post data
        post_info = {
            'title': post.title,
            'author': str(post.author),
            'score': upvotes,
            'text_body': post.selftext if post.is_self else post.url,
            'num_comments': num_comments,
            'comments': {}
        }

        # Fetch comments
        post.comment_sort = "new"
        try:
            post.comments.replace_more(limit=None)  # Expand hidden comments
            for comment in post.comments.list():
                if comment.body != '[deleted]':
                    post_info['comments'][comment.id] = {
                        'comment': comment.body,
                        'score': comment.score,
                        'parentId': comment.parent_id.split('_')[1] if '_' in comment.parent_id else None
                    }
        except Exception as e:
            print(f"⚠️ Error fetching comments for {post.id}: {e}")

        # Save to categorized flair data
        flair_data.setdefault(flair, {})[post.id] = post_info
        meta_data[post.id] = {
            'flair': flair,
            'keywords': [KEYWORD] if KEYWORD else [],
            'upvotes': upvotes,
            'num_comments': num_comments
        }

        pulled_ids.add(post.id)
        count_posts += 1
        time.sleep(RATE_LIMIT_DELAY)  # Respect rate limits

except Exception as e:
    print(f"🚨 Error encountered: {e}")

# Save collected data
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump({'meta': meta_data, 'flair': flair_data}, f, ensure_ascii=False, indent=4)

# Summary
print(f"\n✅ Scraping Complete!")
print(f"✔️ New Posts Pulled: {count_posts}")
print(f"✔️ Total Posts Stored: {len(pulled_ids)}")
print(f"⚠️ Duplicates Skipped: {count_duplicates}")
print(f"⚠️ Skipped (No Comments): {count_skipped}")


📂 Loaded 0 existing posts.
🚀 Fetching up to 10 posts from r/lonely...

✔️ 1. curious why i feel so unwanted (⬆️ 3 upvotes, 💬 8 comments)
✔️ 2. Everybody loves me buy nobody likes me (⬆️ 7 upvotes, 💬 3 comments)
✔️ 3. 25f feeling so lonely and cried a lil bit (⬆️ 1 upvotes, 💬 7 comments)
✔️ 4. Am I asking for too much (⬆️ 6 upvotes, 💬 3 comments)

✅ Scraping Complete!
✔️ New Posts Pulled: 4
✔️ Total Posts Stored: 4
⚠️ Duplicates Skipped: 0
⚠️ Skipped (No Comments): 6
